## REACT AGENT

##### Load the environment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
tavily_api_key = os.getenv('TAVILY_API_KEY')
model_id = os.getenv('MODEL_ID')
aws_region = os.getenv('AWS_REGION')
bedrock_kb_id = os.getenv('BEDROCK_KB_ID')

##### Model

In [ ]:
from langchain_aws import ChatBedrock
llm = ChatBedrock(max_tokens=8192, model=model_id)

##### Memory

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

##### Tools

In [ ]:
import boto3
from langchain_core.tools import tool

@tool
def retriever_tool(query):
    """Query the knowledge base for information related to Agents and Agentic workflow
    
    Args:
        query: The query string to search for
    """
    bedrock_agent = boto3.client('bedrock-agent-runtime', region_name = aws_region)
    print("QUERYING KB")
    response = bedrock_agent.retrieve_and_generate(
        input={
            "text": query  # Your query text goes here
        },
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseId": bedrock_kb_id,
                "modelArn": model_id,
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            }
        }
    )

    kb_results = response['output']['text']
    return kb_results

In [ ]:
@tool
def blogger_tool(context, language):
    """Ue this tool to write a well formatted blog with a blog title
    
    Args:
        context: The context for the blog
        language: Language to write the blog; choose 'English' as default if not mentioned by user.
    """
    print("WRITING BLOG")
    prompt = f""" Your job is to create a blog title and a detailed multi-paragraph blog with sections from this content: {context} \n\n The Blog has to be written in Language : {language}"""

    final_answer = llm.invoke(prompt)           
    return final_answer.content

In [ ]:
@tool
def translate_tool(blog_content, language):
    """Use this tool to translate the blog_content from english to any other language. The context here is the complete blog written in a different language to the one user asked for.
    
    Args:
        blog_content: The blog to be translated
        language: The language to translate to
    """
    print("BLOG TRANSLATION")
    prompt = f""" Your job is to translate the blog:\n {blog_content}. \n\n Translate to {language} from source language. Make sure to be comprehensive when translating; cover complete context."""

    final_answer = llm.invoke(prompt)           
    return final_answer.content

In [ ]:
tools = [retriever_tool, blogger_tool, translate_tool]

## Build the agent (Prebuilt Langgraph)

In [ ]:
from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    llm, tools=tools, 
    checkpointer=memory
)

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

##### Stream the response

In [ ]:
def print_stream(stream):
    """A utility to pretty print the stream."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage
config = {"configurable": {"thread_id": "42"}}

inputs = {"messages": [HumanMessage("what a blog about the types of agents")]}
print_stream(graph.stream(inputs, config, stream_mode="values"))

In [ ]:
inputs = {"messages": [HumanMessage("Can you translate it to deutsch?")]}
print_stream(graph.stream(inputs, config, stream_mode="values"))

## Build the agent (Custom with Post-processing Node) (Advanced)

In [ ]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from typing import (
    Annotated,
    Sequence,
    TypedDict,
    Dict,
    Tuple
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    """The state of the agent."""
    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]
    formatted_blog: Dict
    
tools_by_name = {tool.name: tool for tool in tools}

# Define our tool node
def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# Define the node that calls the model
def call_model(
    state: AgentState,
    config: RunnableConfig,
):
    # this is similar to customizing the create_react_agent with 'prompt' parameter, but is more flexible
    system_prompt = SystemMessage(
        "You are a helpful AI assistant who can write a blog using knowledge retreived and also translate it when required." 
        "\n Please respond to the users query to the best of your ability using tools made available to you."
        "\n translate_tool should be called only to translate the overall blog and not the conversations. Once translate is called, that will be the last step for that converstaion."
    )
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke([system_prompt] + state["messages"], config)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the conditional edge that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

In [ ]:
from langchain_core.messages import ToolMessage, HumanMessage, SystemMessage,AIMessage
from pydantic import BaseModel, Field

class FormattedResponse(BaseModel):
    """Respond to the user in this format."""
    Blog_Title: str = Field(description="title of the blog")
    Blog_Content: str = Field(description="content of the blog")
    Blog_language: str = Field(description="language of the blog")

def formatter_node(state):
    """
    Post-processing node that formats the final blog output using structured LLM parsing.
    Only uses tool outputs from blogger_tool or translate_tool after the most recent human message.
    """
    print("FORMATTING BLOG WITH STRUCTURED PARSER")

    messages = state["messages"]

    # Step 1: Find the index of the most recent HumanMessage
    last_human_index = -1
    human_msg = None
    for i in reversed(range(len(messages))):
        if isinstance(messages[i], HumanMessage):
            last_human_index = i
            human_msg = messages[i]
            break

    if last_human_index == -1:
        print("No recent HumanMessage found. Skipping formatting.")
        return {}

    # Step 2: Gather ToolMessages from blogger_tool or translate_tool after the last HumanMessage
    relevant_content = []
    for msg in messages[last_human_index + 1:]:
        if isinstance(msg, ToolMessage) and msg.name in ["blogger_tool", "translate_tool"]:
            relevant_content.append(msg.content)

    if not relevant_content:
        error_msg = "No recent blogger/translate tool outputs found."
        print(error_msg)
        return {"formatted_blog":{"error":error_msg}}

    combined_blog_text = "\n\n".join(relevant_content)

    # Step 3: Format with structured output
    format_prompt = SystemMessage(
        content=(
            "You will be given a series of tool outputs from a blogging agent as input. The Content can be in any language; depending on whethere translation was performed."
            "Extract or Derive and format it into a JSON object with the following fields:\n"
            "\t- Blog_Title: The title of the blog. Create a title if unable to extract\n"
            "\t- Blog_Content: The content in MD format; Add Headers, Subheaders, Highlighting, Bullets, Numbering wherever required. Use the final language in which blog was written when generating blog_content\n"
            "\t- Blog_language: The language used (detect it)\n\n"
            "Respond only with the Structured format.\n"
        )
    )
    structured_llm = llm.with_structured_output(FormattedResponse)
    final_prompt = [format_prompt, HumanMessage(content=f"Here is the context of the blog for you to provide me with a structured output:\n{combined_blog_text}")]
    result = structured_llm.invoke(final_prompt)
    print(result)
    return {
        "messages":AIMessage("Formatting complete, graph state updated - `formatted_blog`",metadata=result.model_dump()),
        "formatted_blog": result.model_dump()
    }

In [ ]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
# Define the additional node that will be used to format the work after agent finished interactions with tools
workflow.add_node("format_blog",formatter_node)
# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish. (Calling Format_blog is considered last step of agent)
        "end": "format_blog",
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", "agent")
# We now add a normal edge from `format_blog` to `END`.
# This means that after `format_blog` is called, graph workflow is complete.
workflow.add_edge("format_blog", END)

# Now we can compile and visualize our graph
graph = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
def print_stream(stream):
    """A utility to pretty print the stream."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage
config = {"configurable": {"thread_id": "41"}}

inputs = {"messages": [HumanMessage("what a blog about the types of agents")]}
print_stream(graph.stream(inputs, config, stream_mode="values"))

In [ ]:
graph.get_state(config).values.get("formatted_blog")

In [ ]:
inputs = {"messages": [HumanMessage("translate it to french")]}
print_stream(graph.stream(inputs, config, stream_mode="values"))

In [ ]:
graph.get_state(config).values.get("formatted_blog")

In [ ]:
inputs = {"messages": [HumanMessage("traducir el texto al español")]}
print_stream(graph.stream(inputs, config, stream_mode="values"))

In [ ]:
graph.get_state(config).values.get("formatted_blog")